In [1]:
import cv2
import numpy as np
import time
import math

In [2]:
from ipynb.fs.full.HandTrackingModule import HandDetector

In [3]:
class Button():
    def __init__(self,x,y,w,h,value):
        self.x = x
        self.y = y
        self.w = w
        self.h = h
        self.value = value
        
        
    def draw(self,frame):
        cv2.rectangle(frame,(self.x,self.y),(self.x+self.w,self.y+self.h),(0,200,200),-1)
        cv2.rectangle(frame,(self.x,self.y),(self.x+self.w,self.y+self.h),(0,0,0),2)
        if self.value == "space":
            cv2.putText(frame,str(self.value),(self.x+150,self.y+45),0,1,(0,0,0),2)
        elif self.value == "Search":
            cv2.putText(frame,str(self.value),(self.x+10,self.y+45),0,1,(0,0,0),2)    
        else:
            cv2.putText(frame,str(self.value),(self.x+25,self.y+45),0,1,(0,0,0),2)
                
                
    def isClicked(self,frame,x,y):
        if x >= self.x and x <= self.x+self.w and y >= self.y and y <= self.y+self.h :
            return True
        return False
    
    def click(self,frame):
        cv2.rectangle(frame,(self.x,self.y),(self.x+self.w,self.y+self.h),(0,255,255),-1)
        cv2.rectangle(frame,(self.x,self.y),(self.x+self.w,self.y+self.h),(0,0,0),2)
        if self.value == "space":
            cv2.putText(frame,str(self.value),(self.x+150,self.y+45),0,2,(0,0,0),2)
        elif self.value == "Search":
            cv2.putText(frame,str(self.value),(self.x+10,self.y+45),0,2,(0,0,0),2)      
        else:
            cv2.putText(frame,str(self.value),(self.x+25,self.y+45),0,2,(0,0,0),2)
    
    def capitalize(self,frame):
        cv2.putText(frame,str(self.value).upper(),(self.x+25,self.y+45),0,1,(0,0,0),2)
        
    def getValue(self,frame):
        return self.value
    def setValue(self,frame,newValue):
        self.value = newValue

In [11]:
cap = cv2.VideoCapture(0)
cap.set(3,1920)
cap.set(4,720)
detector = HandDetector()
buttonLowerValues = [["q","w","e","r","t","y","u","i","o","p"],
                ["a","s","d","f","g","h","j","k","l"],
                ["z","x","c","v","b","n","m"],
               ["Cap","space","del"]]
buttonUpperValues = [["Q","W","E","R","T","Y","U","I","O","P"],
                     ["A","S","D","F","G","H","J","K","L"],
                     ["Z","X","C","V","B","N","M"],
                    ["Cap","space","del"]]
buttonLowerList = []
buttonUpperList = []
xOffset = 100
yOffset = 200
space = 20
identation = 40
upper = False
delay = 0
mySearchKey = ""
for i in range(len(buttonLowerValues)):
    for j in range(len(buttonLowerValues[i])):
        w,h = 75,75
        x = xOffset + j*(w + space) + identation*i*i
        y = yOffset + i*(h + space)
        if buttonLowerValues[i][j] == "space":
            buttonLowerList.append(Button(375,y,400,h,buttonLowerValues[i][j]))
        elif buttonLowerValues[i][j] == "del":
            buttonLowerList.append(Button(950,y-95,100,h,buttonLowerValues[i][j]))
        elif buttonLowerValues[i][j] == "Cap":
            buttonLowerList.append(Button(100,y-95,100,h,buttonLowerValues[i][j]))
        else:    
            buttonLowerList.append(Button(x,y,w,h,buttonLowerValues[i][j]))

for i in range(len(buttonUpperValues)):
    for j in range(len(buttonUpperValues[i])):
        w,h = 75,75
        x = xOffset + j*(w + space) + identation*i*i
        y = yOffset + i*(h + space)
        if buttonUpperValues[i][j] == "space":
            buttonUpperList.append(Button(375,y,400,h,buttonUpperValues[i][j]))
        elif buttonUpperValues[i][j] == "del":
            buttonUpperList.append(Button(950,y-95,100,h,buttonUpperValues[i][j]))
        elif buttonUpperValues[i][j] == "Cap":
            buttonUpperList.append(Button(100,y-95,100,h,buttonUpperValues[i][j]))
        else:    
            buttonUpperList.append(Button(x,y,w,h,buttonUpperValues[i][j]))            

searchButton = Button(870,100,130,80,"Search")
buttonLowerList.append(searchButton)
buttonUpperList.append(searchButton)            
#print(buttonUpperValues)  
#print(buttonLowerValues)
            
while True:
    success,frame = cap.read()
    if not success:
        break
    frame = cv2.flip(frame,1)
    hands,frame = detector.findHands(frame)
    cv2.rectangle(frame,(100,100),(800,180),(255,255,255),-1)
    #cv2.rectangle(frame,(870,100),(1000,180),(0,200,200),-1)
    #cv2.rectangle(frame,(870,100),(1000,180),(0,0,0),2)
    #cv2.putText(frame,str("Search"),(880,150),4,1,(0,0,0),2)
    
    if not upper:
        for button in buttonLowerList:
            button.draw(frame)
    else:
        for button in buttonUpperList:
            button.draw(frame)
    

    
    
    if hands:
        hand1 = hands[0]
        lmList = hand1["lmList"]
        x1,y1 = lmList[4][0],lmList[4][1]
        x2,y2 = lmList[8][0],lmList[8][1]
        cv2.circle(frame,(x1,y1),10,(255,0,255),-1)
        cv2.circle(frame,(x2,y2),10,(255,0,255),-1)
        cv2.line(frame,(x1,y1),(x2,y2),(255,0,255),2)
        length = math.hypot((x1-x2),(y1-y2))
        if length <= 50 and delay == 0:
            delay = 1
            if upper :
                for button in buttonUpperList:
                    if button.isClicked(frame,x1,y1):
                        button.click(frame)
                        value = button.getValue(frame)
                        #print(value)
                        if value == "Cap":
                            if upper :
                                upper = False
                            else:
                                upper = True
                        elif value == "del":
                            mySearchKey = mySearchKey[:-1]
                        elif value == "space":
                            mySearchKey += " "
                        elif value == "Search":
                            mySearchKey += " "    
                        else:
                            mySearchKey += str(value)
            else:
                for button in buttonLowerList:
                    if button.isClicked(frame,x1,y1):
                        button.click(frame)
                        value = button.getValue(frame)
                        #print(value)
                        if value == "Cap":
                            if upper :
                                upper = False
                            else:
                                upper = True
                        elif value == "del":
                            mySearchKey = mySearchKey[:-1]
                        elif value == "space":
                            mySearchKey += " "
                        elif value == "Search":
                            search(mySearchKey) 
                        else:
                            mySearchKey += str(value)
                            
                            
        if delay != 0:
            delay += 1
            if delay > 10:
                delay = 0
                            
    #cv2.putText(frame,str(delay),(50,50),2,2,(0,0,0),3)                
    cv2.putText(frame,str(mySearchKey),(100,150),2,2,(0,0,0),3)
    cv2.imshow("frame",frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

In [5]:
import webbrowser
def search(keyword):
    try:
        from googlesearch import search
    except ImportError:
        print("No module named 'google' found")
    
    query = keyword
    for j in search(query, tld="co.in", num=10, stop=15, pause=2):
       # print(j)
        if "imdb" in j:
            webbrowser.open(j)  # Go to example.com
            break
            